### 一. 收集器的分类
此前的所有讨论, 都是基于 ` serial collector` ; 事实上1.8的 jvm 包含三种不同的收集器, 各自扮演不同的角色: 
1. The serial collector :   
   仅使用1个线程来完成所有的垃圾回收操作, 因为没有线程间通信的问题, 是的它的十分高效. 特别适合作为单核机器上的 jvm 进程的回收器, 因为它无法利用多核心的优势. 它是为小堆设计的收集器(大约100M) ` -XX:+UseSerialGC` 明确指定

2. The parallel collector (也叫吞吐量收集器) :  
   该收集器可以并行的执行 minor gc, 它是为中等堆到大堆应用设计的收集器. 使用`-XX:+UseParallelGC`打开
    * `Parallel compaction` 是让该收集器可以并行执行 major gc 的新特性. 如果没有该特性,  major gc 也会是单线程的, 将大大影响应用的扩展性. Parallel compaction 特性在开启 `-XX:+UseParallelGC` 后会自动打开, 如果想关掉它可以使用`-XX:-UseParallelOldGC`指定   
    
3. 最大并发(concurrent)的收集器 (小停顿时间) :   
   这种收集器的并行度是最大的, 垃圾收集线程甚至能和应用线程并行, 最大程度的减小 gc 停顿时间. 它专为中型堆和大型堆设计, 且要求应用认为响应时间比吞吐量更重要(因为某些最小化停顿的技术会降低gc的性能). jvm1.8 提供了两种此类收集器 CMS 和 G1. 前者使用`-XX:+UseConcMarkSweepGC`开启; 后者使用`-XX:+UseG1GC`开启

### 二. Selecting a Collector
1. 选择标准 
    * 首先, 如果应用所需的堆非常小, 接近100MB, 使用 Serial 收集器`-XX:+UseSerialGC`  
    * 如果应用跑在单核机器上, 让 jvm 选择用哪个收集器, 或手动指定 Serial 收集器`-XX:+UseSerialGC`
    * 如果应用的性能表现是第一优先级, 并且没有停顿时间要求或是可以接受1秒甚至更长的停顿时间, 可以让 jvm 选择用哪个收集器, 或手动指定 parallel 收集器`-XX:+UseParallelGC`  
    * 如果响应时间比总共的吞吐量更重要, 且要求停顿时间小于1秒, 再去选择并发收集器`XX:+UseConcMarkSweepGC`或`-XX:+UseG1GC`. 

 值得注意的是, 当gc收集器的表现未能到达预期时, 首先要尝试的是调整 total heap 和各个 generation 的大小, 如果仍不能到达预期, 再去选择并发( concurrent )收集器来满足停顿时间的目标, 选择并行( parallel )收集器满足吞吐量的需求